In [14]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [15]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [16]:
from __future__ import print_function
from numpy import *
from scipy import *
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import scipy as sp
import warnings
warnings.filterwarnings('ignore')

import seawater
import datetime as dt
""
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.patches as patches
plt.style.use('seaborn-whitegrid')
import netCDF4 as nc
import time
import cmocean as cm
import glob
import sys
sys.path.append('/data/tjarniko/mocsy')
sys.path.append('/data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/CCCmaDEV/CCCma_src')
import mocsy
import CCCma
import CCCma_stations as cs
from matplotlib import reload
import arrow
import gsw
%matplotlib inline

import xarray as xr
from datetime import datetime
from salishsea_tools import grid_tools, viz_tools




In [17]:
tdir = '/data/tjarniko/results/BASERUN_EXP/MAIN/BR_2nd_2015/ncs/'
gridnc = 'SKOG_*_grid_T_20150101-20150101.nc'
carpnc = 'SKOG_*_carp_T_20150101-20150101.nc'

tgrid = glob.glob(tdir+gridnc)
print(tgrid)
grid = nc.Dataset(tgrid[0])

tcarp = glob.glob(tdir+carpnc)
print(tcarp)
carp = nc.Dataset(tcarp[0])

['/data/tjarniko/results/BASERUN_EXP/MAIN/BR_2nd_2015/ncs/SKOG_1d_20150101_20150130_grid_T_20150101-20150101.nc']
['/data/tjarniko/results/BASERUN_EXP/MAIN/BR_2nd_2015/ncs/SKOG_1d_20150101_20150130_carp_T_20150101-20150101.nc']


In [18]:
# start = time.time()
# pHr, OmAr = OmA_3D(grid,carp)
# end = time.time()

# print(end-start)

In [19]:
def make_nclen(start,end,ftype, sdir):
    fn_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ddmmmyy = tdate.format('DDMMMYY').lower()
        ymd = tdate.format('YYYYMMDD')
        nc_sens = '/data/tjarniko/results/BASERUN_EXP/MAIN/' + sdir + '/ncs/SKOG_1d_*'+ ftype +'*' + ymd + '-' + ymd + '.nc'
        tnc_sens = glob.glob(nc_sens)
        fn_ar.append(tnc_sens[0])

    return fn_ar

def make_fname_ar(start,end,runtype):
    fn_ar = []
    start_run = arrow.get(start)
    end_run = arrow.get(end)
    arrow_array = []
    for r in arrow.Arrow.span_range('day', start_run, end_run):
        arrow_array.append(r)

    dayslen = len(arrow_array)
    for i in range(0,dayslen):
        tdate = arrow_array[i][0]
        ymd = tdate.format('YYYYMMDD')
        fname = runtype + '_OmA_pH_' + ymd +'.nc'
        fn_ar.append(fname)
    
    return fn_ar

def OmA_3D(grid,carp):
    tsal = grid.variables['vosaline'][0,:,:,:]
    ttemp = grid.variables['votemper'][0,:,:,:]
    tdic = carp.variables['dissolved_inorganic_carbon'][0,:,:,:]
    tta = carp.variables['total_alkalinity'][0,:,:,:]

    tsra = np.ravel(tsal)
    ttera = np.ravel(ttemp)
    ttara = np.ravel(tta) * 1e-3
    tdra = np.ravel(tdic) * 1e-3
    tzero = np.zeros_like(tsra)
    tpressure = np.zeros_like(tsra)
    tpressure[:] =1
    tzero = tpressure * 0 
        
    tsra_psu = tsra*35/35.16504
    ttera_is = gsw.t_from_CT(tsra,ttera,tzero)

    response_tup = mocsy.mvars(temp=ttera_is, sal=tsra_psu, alk=ttara, dic=tdra, 
                       sil=tzero, phos=tzero, patm=tpressure, depth=tzero, lat=tzero, 
                        optcon='mol/m3', optt='Tinsitu', optp='m',
                        optb = 'l10', optk1k2='m10', optkf = 'dg', optgas = 'Pinsitu')
    pH,pco2,fco2,co2,hco3,co3,OmegaA,OmegaC,BetaD,DENis,p,Tis = response_tup

    pHr = pH.reshape(40,898,398)
    OmAr = OmegaA.reshape(40,898,398)
    
    return pHr, OmAr

In [24]:
#get a list of filenames for the LA, BR, PI cases
#make a list of .nc names for the new files
#for each day, do calculation and save appropriate file

sdir = 'BR_2nd_2015'
runtype = 'BR'


start = '2015-01-01'
end = '2015-12-31'

carp_ar = make_nclen(start,end,'carp_T', sdir)
grid_ar = make_nclen(start,end,'grid_T', sdir)
fn_ar = make_fname_ar(start,end,runtype)

for i in range(0,2):
    
    tcarp = carp_ar[i]
    print(tcarp)
    carp = nc.Dataset(tcarp)
    tgrid = grid_ar[i]
    grid = nc.Dataset(tgrid)
    fn = fn_ar[i]
    
    pHr, OmAr = OmA_3D(grid,carp)

    tdir = '/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/'
    ncname = tdir + fn

    f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
    g = f.createGroup('model_output')
    g.createDimension('depths', 40)
    g.createDimension('ydir',898)
    g.createDimension('xdir',398)

    ts = g.createVariable('pHr','f4',('depths','ydir','xdir'))
    ts[:] = pHr
    ts2 = g.createVariable('OmAr','f4',('depths','ydir','xdir'))
    ts2[:] = OmAr

    f.close()

/data/tjarniko/results/BASERUN_EXP/MAIN/BR_2nd_2015/ncs/SKOG_1d_20150101_20150130_carp_T_20150101-20150101.nc
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    name: SKOG_1d_20150101_20150130_grid_T
    description: ocean T grid variables
    title: ocean T grid variables
    Conventions: CF-1.6
    timeStamp: 2019-May-07 16:48:45 GMT
    uuid: 547ae1ed-bb29-43d3-86ec-7c8e45f827f3
    history: Wed May 15 17:31:17 2019: ncks -4 -L4 -O /data/tjarniko/results/BR_2nd_2015_cop/SKOG_2/ncs/SKOG_1d_20150101_20150130_grid_T_20150101-20150101.nc /data/tjarniko/results/BR_2nd_2015_cop/SKOG_2/ncs/SKOG_1d_20150101_20150130_grid_T_20150101-20150101.nc
    NCO: "4.5.4"
    dimensions(sizes): y(898), x(398), nvertex(4), deptht(40), axis_nbounds(2), time_counter(1)
    variables(dimensions): float32 area(y,x), float32 bounds_nav_lat(y,x,nvertex), float32 bounds_nav_lon(y,x,nvertex), float32 deptht(deptht), float32 deptht_bounds(deptht,axis_nbounds), float32 nav_l

In [30]:

w = nc.Dataset('/data/tjarniko/results/BASERUN_EXP/MAIN/OmA_pH_calculated/'+ 'BR_OmA_pH_20150101.nc')
print(w['model_output']['OmAr'][0,250,250])




1.0997624
